In [1]:
%pip install pandas
import pandas as pd

%pip install numpy
import numpy as np

%pip install requests
import requests

import sqlite3
from datetime import datetime

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Alerta

In [2]:
%pip install plyer
from plyer import notification

def alerta(nivel, base, etapa):
    if etapa == "inicio": # Para o incio do processo da pipeline
        title = "Pipeline ETL Iniciado"
        message = "Iniciando"

    elif etapa == "fim": # Para o fim do processo da pipeline
        title = "Pipeline ETL Concluído"
        message = "Finalizado"
        
    elif nivel == 0:
        title = "Etapa Concluída"
        message = f"{etapa} da {base} concluída com sucesso."
        
    elif nivel == 1:
        title = "Alerta Baixo"
        message = f"Falha no carregamento {base} na etapa {etapa}"

    elif nivel == 2:
        title = "Alerta Moderado"
        message = f"Falha no carregamento {base} na etapa {etapa}"
    
    elif nivel == 3:
        title = "Alerta Alto"
        message = f"Falha no carregamento {base} na etapa {etapa}"

    else:
        title = "Alerta desconhecido" #Para situaçao de nivel de alerta fora do proposto.
        message = f"Nivel de alerta desconhecido ({nivel}) para {base} na etapa {etapa}"

    notification.notify(
        title=title,
        message=message,
        app_name = "Projeto Final",
        timeout=10
    )


#adicionar a funcao alerta(nivel, base, etapa) em cada etapa

Note: you may need to restart the kernel to use updated packages.


### 1 Extração API

In [3]:
#def get_json_api(url): #funçao comentada para rodar automaticamente. Alterar para a funçao ser chamada em parte do codigo
    #'''
    #    Request GET url e retorna json de saida
    #'''
etapa = "Extraçao"
base = "API"

try:
    response = requests.get(url = "https://brasilapi.com.br/api/banks/v1")

    if response.status_code == 200:
        data_json = response.json()
        alerta(0,base,etapa) #Sucesso na etapa de extracao da API
        data_json
        #return data_json #caso inserido como funçao
    
    # Erro
    else:
        alerta(2,base,etapa) #Falha na etapa de extracao da API
        raise Exception(f"erro request,  {response.status_code} - {url}\n{response.text}")

except Exception as e:
    alerta(3,base,etapa) #erro critico na etapa
    raise e
    

### 1.1 Base Brasil Banks_url

In [4]:
df = pd.DataFrame()
url = "https://brasilapi.com.br/api/banks/v1"
response = requests.get (url)
response.status_code

200

In [5]:
if response.status_code == 200:
    data_json = response.json()
else: 
    print ("erro ao acessar a API")

In [6]:
#data_json

In [7]:
banks_full = pd.DataFrame(data_json)
#banks_full.head(60)

banks_full.columns

Index(['ispb', 'name', 'code', 'fullName'], dtype='object')

In [10]:
ispb = [ispb for ispb in banks_full['ispb']]
name =[name for name in banks_full['name']]
Code = [code for code in banks_full['code']]
fulname = [fn for fn in banks_full ['fullName']]
fulname = [fn for fn in banks_full ['fullName']]

### Tabela 1 - ISPB E NAME

In [11]:
base = "tabelas ISPB e Name"
etapa = "Extraçao"

try:
    banks_full= pd.DataFrame ({'ispb':ispb, 'name': name})
    banks_full = pd.DataFrame (banks_full)
    
    alerta(0,base,etapa) # Sucesso na etapa de extracao

    #banks_full
    #banks_full.head(60)
except Exception as e:
    alerta(2,base,etapa) #falha na extraçao

### TABELA 2 - CODE & FULLNAME

In [12]:
base = "tabelas Code e FullName"
etapa = "Extraçao"

try:
    banks_full= pd.DataFrame ({'code':Code, 'fulname':fulname})
    banks_full = pd.DataFrame (banks_full)
    alerta(0,base,etapa) # Sucesso na etapa de extracao

    #banks_full
    #banks_full.head(60)

except Exception as e:
    alerta(2,base,etapa) #falha na extraçao


### TABELA 3 - ISPB & CODE

In [13]:
base = "tabelas ISPB e Code"
etapa = "Extraçao"

try:
    banks_full= pd.DataFrame ({'ispb':ispb, 'code': Code})
    banks_full = pd.DataFrame (banks_full)
    alerta(0,base,etapa) # Sucesso na etapa de extracao

    #banks_full
    #banks_full.head(60)
    
except Exception as e:
    alerta(2,base,etapa) #falha na extraçao


In [14]:

conn = sqlite3.connect('Projeto.db')

banks_full.to_sql('banks_full', conn, if_exists='replace', index=False)

conn.close()
